<a href="https://colab.research.google.com/github/chloebh9/Curse-detection-data/blob/master/Curse_detection_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd "/content/drive/My Drive/Curse-detection-data"

/content/drive/My Drive/Curse-detection-data


In [5]:
!git clone https://github.com/chloebh9/Curse-detection-data.git

Cloning into 'Curse-detection-data'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 4), reused 2 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 253.84 KiB | 1.68 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [6]:
!pip install transformers torch datasets scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

데이터 전처리

In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

file_path = '/content/drive/MyDrive/Curse-detection-data/Curse-detection-data/dataset.txt'

# 데이터 로드
with open(file_path, 'r', encoding='utf-8') as f:
    data = f.readlines()

# rsplit 후 앞의 두 개 요소만 사용하고, 나머지는 무시
data = [line.strip().rsplit('|', maxsplit=1) for line in data if '|' in line]
df = pd.DataFrame(data, columns=['text', 'label'])
df['label'] = df['label'].astype(int)

# 간단한 전처리 함수 정의 (불용어 제거, 특수문자 제거 등)
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # 다중 공백 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    return text

df['text'] = df['text'].apply(preprocess_text)

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

BERT 토크나이저와 입력 준비

In [2]:
from transformers import BertTokenizer
import torch

# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 데이터셋을 토큰화된 형태로 변환
def tokenize_data(texts, labels, max_length=128):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    inputs['labels'] = torch.tensor(labels)
    return inputs

train_data = tokenize_data(X_train.tolist(), y_train.tolist())
test_data = tokenize_data(X_test.tolist(), y_test.tolist())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

데이터 로더 생성

In [3]:
from torch.utils.data import DataLoader, TensorDataset

# TensorDataset 생성
train_dataset = TensorDataset(train_data['input_ids'], train_data['attention_mask'], train_data['labels'])
test_dataset = TensorDataset(test_data['input_ids'], test_data['attention_mask'], test_data['labels'])

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

BERT 모델 학습

In [4]:
import torch
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# BERT 모델 로드
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 옵티마이저와 스케줄러 설정
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # 에포크 수
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# 학습 루프
model.train()
for epoch in range(3):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f'Epoch {epoch + 1} completed')

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 completed
Epoch 2 completed
Epoch 3 completed


모델 평가

In [5]:
from sklearn.metrics import accuracy_score, classification_report

# 평가 모드로 전환
model.eval()

# 예측 및 평가
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# 결과 출력
accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 0.8171673819742489
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86       733
           1       0.78      0.70      0.74       432

    accuracy                           0.82      1165
   macro avg       0.81      0.79      0.80      1165
weighted avg       0.82      0.82      0.81      1165

